In [261]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import tensorflow as tf
from  sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn.linear_model import Ridge,Lasso,LogisticRegression
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import classification_report,confusion_matrix,roc_auc_score

In [262]:
df=pd.read_csv("Train_data_players.csv",index_col=0)
dft=pd.read_csv("Test_data_player.csv",index_col=0)

In [263]:
dft_sub=pd.read_csv("Test_data_player.csv")

In [264]:
display(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 195 entries, 246048 to 348232
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Batting Avg.        195 non-null    object
 1   Strike rate         195 non-null    int64 
 2   L/R Handed Batsman  195 non-null    object
 3   Wicket keeping      195 non-null    int64 
 4   economy             195 non-null    int64 
 5   avg speed           195 non-null    int64 
 6   fitness index       195 non-null    int64 
 7   Sixes               195 non-null    int64 
 8   fours               195 non-null    int64 
 9   Fast bowling        195 non-null    int64 
 10  Leg Spin            195 non-null    int64 
 11  Off Spin            195 non-null    int64 
 12  against fast bowl   195 non-null    int64 
 13  against leg spin    195 non-null    int64 
 14  against off spin    195 non-null    int64 
 15  Fanbase             195 non-null    int64 
 16  Catch percentage  

None

,Batting Avg.,Strike rate,L/R Handed Batsman,Wicket keeping,economy,avg speed,fitness index,Sixes,fours,Fast bowling,Leg Spin,Off Spin,against fast bowl,against leg spin,against off spin,Fanbase,Catch percentage,Region,Not selected
player ID,,,,,,,,,,,,,,,,,,,
246048,1,4,2,6,5,6,6,6,6,6,5,6,7,7,5,7,6,RGB7,0
387987,1,2,1,5,7,10,10,6,2,7,3,7,8,9,6,6,9,RGB10,1
788442,3,4,1,5,7,2,5,3,5,5,5,3,7,7,5,7,7,RGB6,0
465918,0,5,1,7,5,6,5,9,4,6,5,8,7,9,5,7,6,RGB8,0
295804,0,5,1,6,7,7,7,9,5,7,7,7,8,8,8,9,5,RGB1,0


In [241]:
df["Region"].value_counts()

RGB8     25
RGB6     22
RGB9     20
RGB7     20
RGB2     19
RGB5     19
RGB3     18
RGB1     18
RGB10    17
RGB4     17
Name: Region, dtype: int64

In [242]:
df.groupby(by="Region").sum()["Not selected"].sort_values()

Region
RGB3     0
RGB4     0
RGB1     2
RGB10    2
RGB2     2
RGB7     2
RGB9     2
RGB5     5
RGB6     7
RGB8     7
Name: Not selected, dtype: int64

In [265]:
df["L/R Handed Batsman"].replace("na",np.nan,inplace=True)
df["L/R Handed Batsman"].replace("*",np.nan,inplace=True)
df["L/R Handed Batsman"].value_counts(dropna="False")

1    122
2     69
Name: L/R Handed Batsman, dtype: int64

In [266]:
df["L/R Handed Batsman"].fillna(method="bfill",inplace=True)
df["L/R Handed Batsman"].value_counts(dropna="False")
df["L/R Handed Batsman"]=df["L/R Handed Batsman"].astype("int64")

In [267]:
df["L/R Handed Batsman"].value_counts(dropna="False")

1    123
2     72
Name: L/R Handed Batsman, dtype: int64

In [268]:
df["Batting Avg."].value_counts(dropna=False)
df["Batting Avg."].replace("na","0",inplace=True)
df["Batting Avg."].value_counts(dropna=False)
df["Batting Avg."]=df["Batting Avg."].astype("int64")

In [269]:
df["Batting Avg."].value_counts()

1    45
6    43
0    42
2    35
3    27
4     3
Name: Batting Avg., dtype: int64

In [270]:
def convert(x):
    y=x.replace("RGB","")
    return int(y)

In [271]:
df["Region"]=df["Region"].apply(convert)

In [272]:
cols=list(df.columns)
for i in cols:
    df[i].replace(-3,0,inplace=True)
    df[i].replace(-1,0,inplace=True)

In [273]:
df.columns

Index(['Batting Avg.', 'Strike rate', 'L/R Handed Batsman', 'Wicket keeping',
       'economy', 'avg speed', 'fitness index', 'Sixes', 'fours',
       'Fast bowling', 'Leg Spin', 'Off Spin', 'against fast bowl',
       'against leg spin', 'against off spin', 'Fanbase', 'Catch percentage',
       'Region', 'Not selected'],
      dtype='object')

In [274]:
for i in list(df.columns):
    print(i)
    print(df[i].value_counts())

Batting Avg.
1    45
6    43
0    42
2    35
3    27
4     3
Name: Batting Avg., dtype: int64
Strike rate
4    60
5    51
3    47
2    35
1     2
Name: Strike rate, dtype: int64
L/R Handed Batsman
1    123
2     72
Name: L/R Handed Batsman, dtype: int64
Wicket keeping
7     37
8     34
6     27
5     27
9     25
10    20
4     13
2      6
0      4
3      2
Name: Wicket keeping, dtype: int64
economy
5     37
6     34
7     30
8     28
10    22
9     16
4     12
0      6
2      5
3      4
1      1
Name: economy, dtype: int64
avg speed
6     41
7     40
8     38
10    19
9     19
5     16
4     11
2      5
3      3
0      3
Name: avg speed, dtype: int64
fitness index
8     43
7     38
5     35
10    23
6     23
9     14
4      8
0      5
3      2
2      2
1      2
Name: fitness index, dtype: int64
Sixes
8     49
7     35
6     27
5     25
9     18
10    15
0     11
4      8
3      5
2      2
Name: Sixes, dtype: int64
fours
6     41
5     39
8     26
7     23
4     16
9     15
10    11
3  

In [275]:
df=pd.concat([df,pd.get_dummies(df["Region"],drop_first=True,prefix_sep="-")],axis=1)
df.drop("Region",axis=1,inplace=True)

In [276]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(["Not selected"],axis=1),df["Not selected"], test_size=0.33, random_state=12)

In [255]:
xgb=XGBClassifier()
xgb.fit(X_train,y_train)
pred=xgb.predict(X_test)
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))
xgb.score(X_test,y_test)
print(mean_absolute_error(y_test,pred))
print(roc_auc_score(y_test,pred))

              precision    recall  f1-score   support

           0       0.85      0.98      0.91        54
           1       0.67      0.18      0.29        11

    accuracy                           0.85        65
   macro avg       0.76      0.58      0.60        65
weighted avg       0.82      0.85      0.81        65

[[53  1]
 [ 9  2]]
0.15384615384615385
0.5816498316498318


In [256]:
dft=preprocess(dft)
xgb_pred=xgb.predict(dft)
xgb_pred


array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0], dtype=int64)

In [217]:
lr=LogisticRegression()
lr.fit(X_train,y_train)
pred=lr.predict(X_test)
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))
print(lr.score(X_test,y_test))
print(roc_auc_score(y_test,pred))

              precision    recall  f1-score   support

           0       0.84      0.96      0.90        54
           1       0.33      0.09      0.14        11

    accuracy                           0.82        65
   macro avg       0.59      0.53      0.52        65
weighted avg       0.75      0.82      0.77        65

[[52  2]
 [10  1]]
0.8153846153846154
0.5269360269360269


C:\Users\soory\.conda\envs\tensorflow\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [278]:
def preprocess(df):
    df["L/R Handed Batsman"].replace("na",np.nan,inplace=True)
    df["L/R Handed Batsman"].replace("*",np.nan,inplace=True)
    df["L/R Handed Batsman"].value_counts(dropna="False")
    df["L/R Handed Batsman"].fillna(method="bfill",inplace=True)
    df["L/R Handed Batsman"].value_counts(dropna="False")
    df["L/R Handed Batsman"]=df["L/R Handed Batsman"].astype("int64")
    df["Batting Avg."].value_counts(dropna=False)
    df["Batting Avg."].replace("na","0",inplace=True)
    df["Batting Avg."].value_counts(dropna=False)
    df["Batting Avg."]=df["Batting Avg."].astype("int64")
    df["Region"]=df["Region"].apply(convert)
    cols=list(df.columns)
    for i in cols:
        df[i].replace(-3,0,inplace=True)
        df[i].replace(-1,0,inplace=True)
    df=pd.concat([df,pd.get_dummies(df["Region"],drop_first=True,prefix_sep="-")],axis=1)
    df.drop("Region",axis=1,inplace=True)

    return df

In [279]:
dft=pd.read_csv("Test_data_player.csv",index_col=0)

In [220]:
dft.head()

,Batting Avg.,Strike rate,L/R Handed Batsman,Wicket keeping,economy,avg speed,fitness index,Sixes,fours,Fast bowling,Leg Spin,Off Spin,against fast bowl,against leg spin,against off spin,Fanbase,Catch percentage,Region
player ID,,,,,,,,,,,,,,,,,,
943412,0,5,1,5,3,8,10,8,5,8,8,4,10,10,9,10,6,RGB6
891010,0,5,1,5,5,6,5,5,5,5,6,5,4,6,6,5,5,RGB10
232982,2,3,1,8,6,8,6,4,5,3,5,8,7,4,7,9,5,RGB7
762647,3,4,1,8,6,8,8,7,6,6,6,7,7,8,7,7,7,RGB6
882052,6,4,2,9,6,7,7,9,4,3,7,8,10,9,7,8,6,RGB3


In [280]:
dft=preprocess(dft)

In [222]:
pred=xgb.predict(dft)

In [223]:
pred

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0], dtype=int64)

In [224]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

## Best Model is the Decision Tree as it has the highest recall value for 1

In [277]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
pred=rf.predict(X_test)
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))
rf.score(X_test,y_test)
print(mean_absolute_error(y_test,pred))
print(roc_auc_score(y_test,pred))

dt=DecisionTreeClassifier()
dt.fit(X_train,y_train)
pred=dt.predict(X_test)
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))
dt.score(X_test,y_test)
print(mean_absolute_error(y_test,pred))
print(roc_auc_score(y_test,pred))

              precision    recall  f1-score   support

           0       0.83      1.00      0.91        54
           1       0.00      0.00      0.00        11

    accuracy                           0.83        65
   macro avg       0.42      0.50      0.45        65
weighted avg       0.69      0.83      0.75        65

[[54  0]
 [11  0]]
0.16923076923076924
0.5
              precision    recall  f1-score   support

           0       0.88      0.81      0.85        54
           1       0.33      0.45      0.38        11

    accuracy                           0.75        65
   macro avg       0.61      0.63      0.62        65
weighted avg       0.79      0.75      0.77        65

[[44 10]
 [ 6  5]]
0.24615384615384617
0.6346801346801347


C:\Users\soory\.conda\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [281]:
pred=dt.predict(dft)

In [282]:
pred

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0], dtype=int64)

In [283]:
output = pd.DataFrame({'player ID':dft_sub['player ID'],'Not selected':pred})
output.to_csv('dt.csv',index = False)